#Note

The Folder my_dataset (first version ) can be downloaded from kaggle by the command:

!kaggle datasets download -d abdullatifhalabi/defdo-dataset

Read the Data preparations to know what it contains and how it was created

#Importing libraries

In [1]:
import os
import shutil
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
import cv2
import seaborn as sns
import pickle
import json

#Data preparations

In [ ]:
#Making the main dataset directory
os.mkdir('my_dataset')
os.mkdir('my_dataset/images')
os.mkdir('my_dataset/labels')
# os.mkdir('my_dataset/images/train')
# os.mkdir('my_dataset/images/val')
os.mkdir('my_dataset/labels/train')
os.mkdir('my_dataset/labels/val')

##Car_Plates dataset

In [ ]:
# shutil.rmtree('/content/car_plates')

In [ ]:
# os.mkdir('car_plates')
# os.mkdir('car_plates/labels')
# os.mkdir('car_plates/images')
# # os.mkdir('car_plates/images/train')
# # os.mkdir('car_plates/images/val')
# os.mkdir('car_plates/labels/train')
# os.mkdir('car_plates/labels/val')

###Making a copy of the images directly in colab session

In [ ]:
# Source directory
src_dir0 = '/content/drive/MyDrive/DEfDO_Project/copy_Car_plates/imgs/train'
src_dir1 = '/content/drive/MyDrive/DEfDO_Project/copy_Car_plates/imgs/val'

srcs = [src_dir0 , src_dir1]

# Destination directory
dst_dir0 = '/content/my_dataset/images/train'
dst_dir1 = '/content/my_dataset/images/val'

dst_dirs = [dst_dir0 , dst_dir1]

# Iterate over source and destination directories in parallel
for src, dst in zip(srcs, dst_dirs):
  shutil.copytree(src, dst)

###Function to parse data from xml annotation files

In [ ]:
import xml.etree.ElementTree as ET

def parse_xml(xml_file):
  """Parses an XML file and extracts information.

  Args:
    xml_file: Path to the XML file.

  Returns:
    A dictionary containing extracted information.
  """

  tree = ET.parse(xml_file)
  root = tree.getroot()

  data = {
      'filename': root.find('filename').text,
      'width': int(root.find('size/width').text),   #Width of the whole image
      'height': int(root.find('size/height').text),   #Height of the whole image
      #Dimensions on the bouding box
      'xmin' : int(root.find('object/bndbox/xmin').text),
      'ymin' : int(root.find('object/bndbox/ymin').text),
      'xmax' : int(root.find('object/bndbox/xmax').text),
      'ymax' : int(root.find('object/bndbox/ymax').text)
      }

  return data



###Making txt files for annotations as YOLO requires

In [ ]:
car_plates = '/content/drive/MyDrive/DEfDO_Project/copy_Car_plates/annot'

modes = ['/train' , '/val']
for mode in modes:
  for fi in os.listdir(car_plates+ mode):
    data = parse_xml(car_plates+ mode +'/'+ fi)
    xc = round((data['xmin'] + data['xmax']) / (2 * data['width']) , 5)
    yc = round((data['ymin'] + data['ymax']) / (2 * data['height']) , 5)
    bw = round((data['xmax'] - data['xmin']) / data['width'] , 5)   #box_width
    bh = round((data['ymax'] - data['ymin']) / data['height'] , 5)  #box_height

    with open('/content/my_dataset/labels/' + mode + '/' + data['filename'][:-4] + '.txt', 'w') as f:
      f.write(f'80 {xc} {yc} {bw} {bh}')

## Person, car, traffic light  datasets

###Loading the data

In [ ]:
from IPython.display import clear_output

!pip install fiftyone
clear_output()

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

dataset = foz.load_zoo_dataset(
    "coco-2017",
    splits=['train', 'validation'],
    label_types="detections",
    classes=["person", "car", 'traffic light'],
    max_samples=2000,
    only_matching=True,
    dataset_name="my_dataset",
    shuffle=True
)

# session = fo.launch_app(dataset)


INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/coco-2017/train' if necessary


INFO:fiftyone.utils.coco:Downloading annotations to '/root/fiftyone/coco-2017/tmp-download/annotations_trainval2017.zip'


 100% |██████|    1.9Gb/1.9Gb [3.0s elapsed, 0s remaining, 665.8Mb/s]       


INFO:eta.core.utils: 100% |██████|    1.9Gb/1.9Gb [3.0s elapsed, 0s remaining, 665.8Mb/s]       


Extracting annotations to '/root/fiftyone/coco-2017/raw/instances_train2017.json'


...

INFO:fiftyone.zoo.datasets:Loading 'coco-2017' split 'validation'


 100% |█████████████████| 265/265 [2.7s elapsed, 0s remaining, 122.4 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 265/265 [2.7s elapsed, 0s remaining, 122.4 samples/s]      


Dataset 'my_dataset' created


INFO:fiftyone.zoo.datasets:Dataset 'my_dataset' created


In [ ]:
# Moving the folder fiftyone from ''/root' to '/content'
!mv '/root/fiftyone' '/content'


In [ ]:
#removing unecessary files for sake of storage
shutil.rmtree('/content/fiftyone/coco-2017/raw')

###Functions

In [ ]:
def imagesInfoFromJson(json_file):

 '''
 A function to make a dictionnary with image's id as key and
  its necessary info in a dictionary format as value,
    example:
    { 87038: { "file_name": "000000087038.jpg",
                "height": 480,
                "width": 640,
              },...
      }
 Args:
    json_file: Path to the COCO-formatted json file for detection.

  Returns:
    A dictionary containing extracted information.

 '''
 with open(json_file, 'r') as f:
    labels = json.load(f)

 images = {}
 for img in labels['images']:
    id = img['id']
    keys_to_remove = ['id','license', 'date_captured', 'coco_url', 'flickr_url']
    for key in keys_to_remove:
      del img[key]

    images[id] = img

 return images



{483108: {'file_name': '000000483108.jpg', 'height': 640, 'width': 428},
 515289: {'file_name': '000000515289.jpg', 'height': 640, 'width': 639},
 562150: {'file_name': '000000562150.jpg', 'height': 516, 'width': 640},
 412151: {'file_name': '000000412151.jpg', 'height': 334, 'width': 500},
 509822: {'file_name': '000000509822.jpg', 'height': 425, 'width': 640},
 450263: {'file_name': '000000450263.jpg', 'height': 480, 'width': 640},
 455859: {'file_name': '000000455859.jpg', 'height': 480, 'width': 640},
 155885: {'file_name': '000000155885.jpg', 'height': 346, 'width': 500},
 236182: {'file_name': '000000236182.jpg', 'height': 640, 'width': 480},
 551952: {'file_name': '000000551952.jpg', 'height': 480, 'width': 640},
 400: {'file_name': '000000000400.jpg', 'height': 640, 'width': 638},
 350694: {'file_name': '000000350694.jpg', 'height': 336, 'width': 500},
 246809: {'file_name': '000000246809.jpg', 'height': 333, 'width': 500},
 412914: {'file_name': '000000412914.jpg', 'height': 3

In [ ]:
def create_and_write_first_line(file_path, first_line):
  """Creates a file and writes the specified content as the first line.

  Args:
    file_name: The path of the file to create.
    first_line: The content to write as the first line.
  """

  with open(file_path, 'w') as file:
    file.write(first_line)


#############################################################


def append_to_file(file_path, line_to_append):
  """Appends a line to an existing file.

  Args:
    file_name: The path of the file to append to.
    line_to_append: The line to append to the file.
  """

  with open(file_path, 'a') as file:
    file.write(f"\n{line_to_append}")


In [ ]:
def fromJsonToTxt(json_file, destination, max_nb_per_class, catg_ids):
  '''A functtion to transform labels from json to txt files as YOLO requires.

  Args:
    json_file(str): The path to the json file
    destination(str): The path to where to save the txt files.(labels)
    max_nb_per_class(int): The maximum number of bounding boxes can a category have.
                      Useful when the desried categories have varying amounts of images.
    catg_ids(list): A list of integers representing the IDs of the desired categories.(Look at the Json file)

  Returns:
    None
  '''

  print('Collecting necessary informations of the images ...')

  images_info = imagesInfoFromJson(json_file)
  print('Collected successfully !')

  with open(json_file, 'r') as f:
    labels = json.load(f)

  annotations = []
  img_amount = {key:0 for key in catg_ids}
  imgs ={}

  print('Choosing the desired categories ...\n Setting the maximum number of bounding boxes can a category have...')
  for annot in labels['annotations']:
    if annot['category_id'] in catg_ids and img_amount[annot['category_id']] < max_nb_per_class:

      keys_to_remove = ['segmentation', 'area', 'iscrowd', 'id']
      for key in keys_to_remove:
        del annot[key]

      w  = images_info[annot['image_id']]['width']                        #image_width
      h  = images_info[annot['image_id']]['height']                       #image_height

      xc = round((annot['bbox'][0] + annot['bbox'][2] / 2) / w , 5)    #x_center_normalized
      yc = round((annot['bbox'][1] + annot['bbox'][3] / 2) / h , 5)    #y_center_normalized
      bw = round( annot['bbox'][2] / w , 5)                            #box_width_normalized
      bh = round( annot['bbox'][3] / h , 5)                            #box_height_normalized
      annot['bbox'] = [xc, yc, bw, bh]
      annotations.append(annot)

      img_amount[annot['category_id']] += 1

      if annot['image_id'] not in imgs:
        imgs[annot['image_id']] = destination + '/' + images_info[annot['image_id']]['file_name'][:-3] + 'txt'                #file_path as value

        file_path = imgs[annot['image_id']]
        first_line = f"{annot['category_id']} {annot['bbox'][0]} {annot['bbox'][1]} {annot['bbox'][2]} {annot['bbox'][3]}"
        create_and_write_first_line(file_path , first_line)

      else:
        line_to_append = f"{annot['category_id']} {annot['bbox'][0]} {annot['bbox'][1]} {annot['bbox'][2]} {annot['bbox'][3]}"
        append_to_file(imgs[annot['image_id']] , line_to_append)

  print(f'the txt labeling files are now at {destination}')
  print(f'the number of boxes per category is {img_amount}')
  print('\n')





In [ ]:
def copy_files(src_dir, dst_dir):
    for filename in os.listdir(src_dir):
        src_path = os.path.join(src_dir, filename)
        dst_path = os.path.join(dst_dir, filename)
        shutil.copy(src_path, dst_path)


###Processing for YOLO requirements

In [ ]:
fromJsonToTxt('/content/fiftyone/coco-2017/train/labels.json', 'my_dataset/labels/train', 600, [1,3,10])
fromJsonToTxt('/content/fiftyone/coco-2017/validation/labels.json', 'my_dataset/labels/val', 250, [1,3,10])

Collected successfully !
Choosing the desired categories ...
 Setting the maximum number of bounding boxes can a category have...
the txt labeling files are now at my_dataset/labels/train
the number of boxes per category is {1: 600, 3: 600, 10: 565}


Collected successfully !
Choosing the desired categories ...
 Setting the maximum number of bounding boxes can a category have...
the txt labeling files are now at my_dataset/labels/val
the number of boxes per category is {1: 250, 3: 250, 10: 96}




In [ ]:
src_train = '/content/fiftyone/coco-2017/train/data'
dst_train = '/content/my_dataset/images/train'
copy_files(src_train, dst_train)

src_val = '/content/fiftyone/coco-2017/validation/data'
dst_val = '/content/my_dataset/images/val'
copy_files(src_val, dst_val)

####Making copy of the directory in drive

In [ ]:

s = '/content/my_dataset'
d = '/content/drive/MyDrive/DEfDO_Project/my_dataset'

shutil.copytree(s, d)

'/content/drive/MyDrive/DEfDO_Project/my_dataset'

#Modeling

##Making  a Yaml file to fine tune


In [ ]:
  # 1: person
  # 3: car
  # 10: traffic light
  # 80: vehicle plate

In [ ]:
%%writefile my_dataset.yaml
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: my_dataset  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
# test:  # test images (optional)

# Classes
nc: 81  # number of classes
names:
  0: bicycle
  1: person
  2: motorcycle
  3: car
  4: airplane
  5: bus
  6: train
  7: truck
  8: boat
  9: fire hydrant
  10: traffic light
  11: stop sign
  12: parking meter
  13: bench
  14: bird
  15: cat
  16: dog
  17: horse
  18: sheep
  19: cow
  20: elephant
  21: bear
  22: zebra
  23: giraffe
  24: backpack
  25: umbrella
  26: handbag
  27: tie
  28: suitcase
  29: frisbee
  30: skis
  31: snowboard
  32: sports ball
  33: kite
  34: baseball bat
  35: baseball glove
  36: skateboard
  37: surfboard
  38: tennis racket
  39: bottle
  40: wine glass
  41: cup
  42: fork
  43: knife
  44: spoon
  45: bowl
  46: banana
  47: apple
  48: sandwich
  49: orange
  50: broccoli
  51: carrot
  52: hot dog
  53: pizza
  54: donut
  55: cake
  56: chair
  57: couch
  58: potted plant
  59: bed
  60: dining table
  61: toilet
  62: tv
  63: laptop
  64: mouse
  65: remote
  66: keyboard
  67: cell phone
  68: microwave
  69: oven
  70: toaster
  71: sink
  72: refrigerator
  73: book
  74: clock
  75: vase
  76: scissors
  77: teddy bear
  78: hair drier
  79: toothbrush
  80: vehicle plate

Writing my_dataset.yaml


##make copies in drive

In [ ]:
# !cp my_dataset.yaml /content/drive/MyDrive/DEfDO_Project


##Training

In [1]:
!pip install ultralytics --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.0/869.0 kB 13.5 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO

In [3]:
%cd /content/drive/MyDrive/DEfDO_Project

/content/drive/MyDrive/DEfDO_Project


In [ ]:
model = YOLO('yolov8s.pt')

100%|██████████| 21.5M/21.5M [00:00<00:00, 96.7MB/s]


In [ ]:
# current directory
!pwd

/content/drive/MyDrive/DEfDO_Project


In [ ]:
# shutil.rmtree('/content/drive/MyDrive/DEfDO_Project/runs')

In [ ]:
results = model.train(data = 'my_dataset.yaml' , epochs = 150, imgsz = 640, batch = 8 )

Ultralytics YOLOv8.2.78 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=my_dataset.yaml, epochs=150, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

100%|██████████| 755k/755k [00:00<00:00, 28.8MB/s]


Overriding model.yaml nc=80 with nc=81

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

100%|██████████| 6.25M/6.25M [00:00<00:00, 101MB/s]


AMP: checks passed ✅


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     47/150      2.49G      1.018     0.8739     0.9881          0        640: 100%|██████████| 279/279 [01:16<00:00,  3.63it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:04<00:00,  5.08it/s]

                   all        340        671      0.515       0.43       0.43      0.253



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     48/150      2.49G      1.058     0.8809     0.9839          6        640:  57%|█████▋    | 159/279 [00:40<00:42,  2.83it/s]

###Resume Training

In [ ]:
model = YOLO('runs/detect/train/weights/last.pt')

results = model.train(data = 'my_dataset.yaml' , epochs = 105, imgsz = 640, batch = 8 )

Ultralytics YOLOv8.2.78 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=runs/detect/train/weights/last.pt, data=my_dataset.yaml, epochs=105, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

100%|██████████| 755k/755k [00:00<00:00, 109MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

100%|██████████| 6.25M/6.25M [00:00<00:00, 173MB/s]


AMP: checks passed ✅


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     55/105      2.49G      0.841     0.6214     0.9168          9        640: 100%|██████████| 279/279 [01:10<00:00,  3.94it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:05<00:00,  3.88it/s]

                   all        340        671      0.539      0.375       0.39      0.238



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     56/105      2.48G     0.8639     0.6208     0.9287          3        640:  39%|███▉      | 110/279 [00:26<00:33,  5.09it/s]

##Validation

In [4]:
best_model = YOLO("runs/detect/train2/weights/best.pt")  # load a custom model

# Validate the model
metrics = best_model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

Ultralytics YOLOv8.2.78 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 11,156,931 parameters, 0 gradients, 28.6 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 15.6MB/s]
val: Scanning /content/drive/MyDrive/DEfDO_Project/my_dataset/labels/val.cache... 172 images, 168 backgrounds, 0 corrupt: 100%|██████████| 340/340 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [05:02<00:00, 13.76s/it]


                   all        340        671      0.553      0.424      0.438       0.26
                person         50        250       0.34      0.076      0.112     0.0693
                   car         61        250      0.645        0.3       0.39      0.256
         traffic light         24         96      0.454       0.45      0.372      0.193
         vehicle plate         75         75      0.775      0.872      0.879      0.521
Speed: 6.7ms preprocess, 827.7ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/val


array([    0.25969,    0.069302,     0.25969,     0.25598,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.19252,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,
           0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,
           0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,     0.25969,
           0.25969,     0.25969,     0.2596

##Testing

In [ ]:
shutil.rmtree('runs/detect/train62')

In [5]:
# prompt: use the model to predict on the file Cars390.png  and show the image with the boundingboxes predicted
from PIL import Image
import glob
from IPython.display import Image, display

tests = best_model('testing', save=True)




image 1/18 /content/drive/MyDrive/DEfDO_Project/testing/Cars357.png: 448x640 1 vehicle plate, 570.5ms
image 2/18 /content/drive/MyDrive/DEfDO_Project/testing/Cars358.png: 320x640 2 vehicle plates, 409.2ms
image 3/18 /content/drive/MyDrive/DEfDO_Project/testing/Cars359.png: 480x640 1 car, 588.3ms
image 4/18 /content/drive/MyDrive/DEfDO_Project/testing/Cars360.png: 384x640 1 vehicle plate, 459.0ms
image 5/18 /content/drive/MyDrive/DEfDO_Project/testing/Cars361.png: 480x640 1 vehicle plate, 552.9ms
image 6/18 /content/drive/MyDrive/DEfDO_Project/testing/Cars362.png: 480x640 (no detections), 538.3ms
image 7/18 /content/drive/MyDrive/DEfDO_Project/testing/Cars363.png: 448x640 1 vehicle plate, 764.3ms
image 8/18 /content/drive/MyDrive/DEfDO_Project/testing/Cars364.png: 416x640 1 vehicle plate, 757.6ms
image 9/18 /content/drive/MyDrive/DEfDO_Project/testing/Cars365.png: 448x640 1 vehicle plate, 756.0ms
image 10/18 /content/drive/MyDrive/DEfDO_Project/testing/Cars366.png: 384x640 1 vehicle pl

In [7]:
image_files = glob.glob('runs/detect/predict/*.png')

for image_file in image_files:
  display(Image(filename=image_file))
  print("\n")

Output hidden; open in https://colab.research.google.com to view.

#Analyses & Conclusion

Our analysis revealed that the model's training encountered difficulties. Both the bounding box and classification losses plateaued during training, indicating suboptimal convergence. While classification loss improved relatively quickly, bounding box loss progressed slowly. These training dynamics, combined with a significant data imbalance, likely contributed to the model's overall performance issues.

The dataset exhibits a substantial imbalance:The 'car_plates' dataset is only focusing on car plates while neglecting annotations for the other objects (people, traffic lights, and cars) even when present in images.

To enhance the model's ability to detect all target objects, manually adding bounding boxes for these objects to the dataset is imperative.